In [105]:
import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [169]:
def concat_dfs(folder_name):
    directory = r'./data/wyniki_gis_udział_luk/{}/*.csv'.format(folder_name)
    files = glob.glob(directory)
    df = []

    for f in files:   
        col_name = f.split('/')[-1].split('.')[0]
        csv = pd.read_csv(f, names = [col_name],  keep_default_na=False)
        csv.reset_index(drop=True, inplace=True)
        df.append(csv)
    
    df2 = pd.concat(df,axis=1)
    return df2.reindex(sorted(df2.columns), axis=1)   

In [199]:
north_10m = pd.melt(concat_dfs('wyniki csv 10m północ'))
south_10m = pd.melt(concat_dfs('wyniki csv 10m południe'))
buffer_10m = pd.melt(concat_dfs('wyniki csv 10m buffer'))

In [215]:
north_2014_2018 = north_10m.loc[(north_10m['variable'].isin(['martwe_drzewa_2014',
                                                            'martwe_drzewa_2015',
                                                            'martwe_drzewa_2016',
                                                            'martwe_drzewa_2017',
                                                            'martwe_drzewa_2018']))].dropna()
north_2019_2021 = north_10m.loc[(north_10m['variable'].isin(['martwe_drzewa_2019',
                                                            'martwe_drzewa_2020',
                                                            'martwe_drzewa_2021']))].dropna()

In [216]:
north_2019_2021

,variable,value
11820,martwe_drzewa_2019,"37,474313138"
11821,martwe_drzewa_2019,"50,9284865494"
11822,martwe_drzewa_2019,"79,7131337932"
11823,martwe_drzewa_2019,"68,9521229762"
11824,martwe_drzewa_2019,"89,1652906673"
...,...,...
16575,martwe_drzewa_2021,"98,7224831701"
16576,martwe_drzewa_2021,"92,6606123447"
16577,martwe_drzewa_2021,"103,2960582"
16578,martwe_drzewa_2021,"5,01248203003"


In [209]:
%matplotlib inline